In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import (upload
)
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,

)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes
logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *


In [ ]:
plate_id = "310_20230830"
plate_id_video = "20230903_Plate310"
videos_folder = "/projects/0/einf914/videos/"


In [ ]:
indexes = {
"20230901_Plate310" : 20,
"20230902_Plate310" : 32,
"20230904_Plate310" : 44,
    
}

In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]

In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data_network.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select_network = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]

In [ ]:
vid_anls_frame_merged = vid_anls_frame_select.merge(vid_anls_frame_select_network[['xpos_network','ypos_network','unique_id']],on = 'unique_id')

In [ ]:
analysis_folder_root = "/projects/0/einf914/analysis_videos/"


In [ ]:
vid_anls_frame_merged["analysis_folder"] = analysis_folder_root
vid_anls_frame_merged["videos_folder"] = [
    str(Path(videos_folder) / entry["folder"])
    for index, entry in vid_anls_frame_merged.iterrows()
]

In [ ]:
data_obj = HighmagDataset(vid_anls_frame_merged, analysis_folder_root, videos_folder)

In [ ]:
# directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
directory_targ = '/projects/0/einf914/transport/'
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]

In [ ]:
folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = 44
exp.load(folders.iloc[i : i + 2], suffix="_width")
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
mpl.rcParams["figure.dpi"] = 50

from random import choice
vid_obj = data_obj.video_objs[15]
vid_obj.plot_speed_arrows(plot_both=True, video_txt_size=30,plot_flux = True,plot_text=False)


In [ ]:
for edge in vid_obj.edge_objs:
    edge.time_data["flux_mean"].mean()
    break
edge.time_data["flux_mean"].mean()
    

In [ ]:
edge = vid_obj.edge_objs[0]

In [ ]:
edge_begin = edge.mean_data["network_begin"]
edge_end = edge.mean_data["network_end"]

In [ ]:
network_edge = Edge(edge_begin,edge_end,exp)

In [ ]:
network_edge